Ok, so we are gonna take the whole dataset and then do a partition, training testing, with 80% training and 20% testing 
to see if we can actually predict the ecotypes frequenci or rank at each location 

In [ ]:
## teh frist one is going to be a leave one out approach for the sites vb

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import pickle
import dask.dataframe as dd

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess
import random

/home/tbellagio/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
## for the final generation we have 31 unique sites
# 31*0.8 is 24.8
## so i am going to be using 25 for trianing and  6 for testing 

In [4]:
with open('splits_samples.pkl', 'rb') as file:
    # Load data from the file
    splits_samples = pickle.load(file)

In [16]:
split_number = 0 

In [29]:
for split_number in range(100):
    train_samples = splits_samples[split_number][0]
    
    test_samples = splits_samples[split_number][1]
    
    # File and directory setup
    directory_name = f"split_number_{split_number}"
    os.makedirs(directory_name, exist_ok=True)  # Creates the directory if it doesn't exist
    
    num_components = 2
    # Open the file in write mode and write the integer with a newline character at the end
    with open(f'split_number_{split_number}/num_components_full_genome.txt', 'w') as file:
        file.write(f"{num_components}\n")
    
    clim_sites_during_exp = pd.read_csv('/carnegie/nobackup/scratch/tbellagio/grene/data/bioclimvars_experimental_sites_era5.csv')
    
    sites_train = pd.Series(train_samples).str.split('_').str[0].astype(int)
    
    sites_train.name = 'site'
    
    sites_test = pd.Series(test_samples).str.split('_').str[0].astype(int)
    
    sites_test.name = 'site'
    
    env_train = sites_train.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)
    env_test = sites_test.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)
    
    ## for now only work with bio1 and bio12 
    env_train = env_train[['site', 'bio1', 'bio12']]
    env_test = env_test[['site', 'bio1', 'bio12']]
    
    ##scale it
    for i in env_train.columns[1:]:
        mean_train = np.mean(env_train[i])
        std_train = np.std(env_train[i])
        env_train[i] = (env_train[i] - mean_train) / std_train
    
        env_test[i] = (env_test[i] - mean_train) / std_train
    
    ## delete the sites column
    env_train = env_train.drop('site',axis=1)
    
    env_train.to_csv(f'split_number_{split_number}/environment_lea_full_genome.csv', sep = ',', index=False)
    
    env_test.to_csv(f'split_number_{split_number}/environment_test.csv', sep = ',', index=False)

In [52]:
path_all_af_indexed = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/baypass_terminal/merged_hapFIRE_allele_frequency_indexed.csv'

In [53]:
columns_right = [i for i in pd.read_csv(path_all_af_indexed,nrows=1, sep=',' ).columns if '_' in i]

In [49]:
path_delta_p_normed = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_hapFIRE_delta_p_normed.txt'

In [50]:
delta_p_normed = dd.read_csv(path_delta_p_normed, sep = '\t')
delta_p_normed = delta_p_normed.compute() 

In [55]:
delta_p_normed.columns = columns_right

In [63]:
delta_p_normed = delta_p_normed.round(6)

In [64]:
#delta_p_normed.to_csv('delta_p_normed.csv',index=None)

In [32]:
!pwd

/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv


In [5]:
wd = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/'

In [6]:


# Assuming splits_samples is defined elsewhere
shfiles = []
for number, (train_samples, test_samples) in enumerate(splits_samples):
    seed = random.randint(1, 100000000)
    file = wd + f'split_{number}.sh'
    train_samples_str = " ".join(map(str, train_samples))  # map to str in case elements are not strings
    # Format the commands with proper handling of list conversion to string
    cmd1 = f"python preproc_lfmm-full_genome.py {number} '{train_samples_str}'"
    cmd2 = f'Rscript run_lfmm_full_genome_ridge.r {number} "{train_samples}"'      # Pass necessary args to R script
    text = f'''#!/bin/bash
#SBATCH --job-name=split_{number}
#SBATCH --time=4:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=120gb
#SBATCH --cpus-per-task=2
#SBATCH --output=split_{number}_%j.out
#SBATCH --mail-user=tbellagio@carnegiescience.edu
#SBATCH --mail-type=FAIL

source /home/tbellagio/miniforge3/etc/profile.d/conda.sh

export PATH="${{PATH}}:/home/username/bin"

cd /carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv

conda activate /home/tbellagio/miniforge3/envs/r-environment
{cmd2}
'''

    with open(file, 'w') as o:
        o.write(text)
    shfiles.append(file)
    
    # Launch the job



In [ ]:
conda activate /home/tbellagio/miniforge3/envs/pipeline_snakemake
{cmd1}
conda deactivate 

In [7]:
shfiles[0]

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_0.sh'

In [8]:
shfiles

['/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_0.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_1.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_2.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_3.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_4.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_5.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_6.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_7.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_8.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_9.sh',
 '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_10.sh',
 '/carneg

In [9]:
subprocess.run(['sbatch', '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_99.sh'])

Submitted batch job 3736


CompletedProcess(args=['sbatch', '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/sh_files/split_99.sh'], returncode=0)

In [10]:
for file in shfiles:
    subprocess.run(['sbatch', file])

Submitted batch job 3737
Submitted batch job 3738
Submitted batch job 3739
Submitted batch job 3740
Submitted batch job 3741
Submitted batch job 3742
Submitted batch job 3743
Submitted batch job 3744
Submitted batch job 3745
Submitted batch job 3746
Submitted batch job 3747
Submitted batch job 3748
Submitted batch job 3749
Submitted batch job 3750
Submitted batch job 3751
Submitted batch job 3752
Submitted batch job 3753
Submitted batch job 3754
Submitted batch job 3755
Submitted batch job 3756
Submitted batch job 3757
Submitted batch job 3758
Submitted batch job 3759
Submitted batch job 3760
Submitted batch job 3761
Submitted batch job 3762
Submitted batch job 3763
Submitted batch job 3764
Submitted batch job 3765
Submitted batch job 3766
Submitted batch job 3767
Submitted batch job 3768
Submitted batch job 3769
Submitted batch job 3770
Submitted batch job 3771
Submitted batch job 3772
Submitted batch job 3773
Submitted batch job 3774
Submitted batch job 3775
Submitted batch job 3776
